# Случайные леса

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с бустингом, предоставляемым библиотекой `CatBoost`.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека.

In [1]:
import inspect
import random
from collections import Counter
from dataclasses import dataclass
from itertools import product
from typing import Callable, List, Tuple, Union

import numpy as np
import numpy.typing as npt
import pandas
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)


# Этой функцией будут помечены все места, которые необходимо дозаполнить
# Это могут быть как целые функции, так и отдельные части внутри них
# Всегда можно воспользоваться интроспекцией и найти места использования этой функции :)
def todo():
    stack = inspect.stack()
    caller_frame = stack[1]
    function_name = caller_frame.function
    line_number = caller_frame.lineno
    raise NotImplementedError(f"TODO at {function_name}, line {line_number}")


SEED = 0xC0FFEE
set_seed(SEED)

In [3]:
def mode(data):
    counts = Counter(data)
    return counts.most_common(n=1)[0][0]

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bootstrap. Элементы, которые не вошли в новую обучающую выборку, образуют **out-of-bag** выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bootstrap. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [4]:
# Для начала реализуем сами критерии


def gini(x: npt.ArrayLike) -> float:
    """
    Calculate the Gini impurity of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Gini impurity value.
    """

    if x.size == 0:
        return 0.0
    
    counts = np.array(list(Counter(x).values()))
    probabilities = counts / counts.sum()
    return np.sum(probabilities * (1 - probabilities))



def entropy(x: npt.ArrayLike) -> float:
    """
    Calculate the entropy of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Entropy value.
    """
    if x.size == 0:
        return 0.0
    
    counts = np.array(list(Counter(x).values()))
    probabilities = counts / counts.sum()
    return -np.sum(probabilities * np.log2(probabilities + 1e-10))

def gain(left_y: npt.ArrayLike, right_y: npt.ArrayLike, criterion: Callable[[npt.ArrayLike], float]) -> float:
    """
    Calculate the information gain of a split using a specified criterion.

    Args:
        left_y (ArrayLike): Class labels for the left split.
        right_y (ArrayLike): Class labels for the right split.
        criterion (Callable): Function to calculate impurity (e.g., gini or entropy).

    Returns:
        float: Information gain from the split.
    """
    n_total = len(left_y) + len(right_y)
    parent = criterion(np.concatenate([left_y, right_y]))
    child = (len(left_y) / n_total) * criterion(left_y) + (len(right_y) / n_total) * criterion(right_y)
    return parent - child

In [5]:
@dataclass
class DecisionTreeLeaf:
    classes: np.ndarray

    def __post_init__(self):
        self.max_class = mode(self.classes)


@dataclass
class DecisionTreeInternalNode:
    split_dim: int
    left: Union["DecisionTreeInternalNode", DecisionTreeLeaf]
    right: Union["DecisionTreeInternalNode", DecisionTreeLeaf]


DecisionTreeNode = Union[DecisionTreeInternalNode, DecisionTreeLeaf]

In [6]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = int(np.sqrt(X.shape[1])) if max_features == "auto" else max_features

        n_samples = X.shape[0]
        bootstrap_indices = np.random.choice(n_samples, n_samples, replace=True)
        
        oob_indices = np.setdiff1d(np.arange(n_samples), np.unique(bootstrap_indices))
        self._out_of_bag_X = X[oob_indices] if len(oob_indices) > 0 else None
        self._out_of_bag_y = y[oob_indices] if len(oob_indices) > 0 else None
        
        self._root = self._build_node(X[bootstrap_indices], y[bootstrap_indices], depth=0)

    @property
    def out_of_bag(self) -> Tuple[np.ndarray, np.ndarray]:
        return self._out_of_bag_X, self._out_of_bag_y

    def _build_node(self, points: np.ndarray, classes: np.ndarray, depth: int) -> DecisionTreeNode:
        if (self.max_depth and depth >= self.max_depth) or \
            (len(classes) < 2 * self.min_samples_leaf) or \
            (len(np.unique(classes)) == 1):
            return DecisionTreeLeaf(classes)
        
        best_gain = -np.inf
        best_split_dim = None
        best_left_mask = None

        features = np.random.choice(points.shape[1], self.max_features, replace=False)
    
        for dim in features:
            left_mask = points[:, dim] == 0
            right_mask = ~left_mask

            if left_mask.sum() < self.min_samples_leaf or right_mask.sum() < self.min_samples_leaf:
                continue
            
            current_gain = gain(classes[left_mask], classes[right_mask], self.criterion)
            
            if current_gain > best_gain:
                best_gain = current_gain
                best_split_dim = dim
                best_left_mask = left_mask

        if best_split_dim is None:
            return DecisionTreeLeaf(classes)
        
        left_node = self._build_node(points[best_left_mask], classes[best_left_mask], depth+1)
        right_node = self._build_node(points[~best_left_mask], classes[~best_left_mask], depth+1)
        
        return DecisionTreeInternalNode(best_split_dim, left_node, right_node)

    def _predict(self, points: np.ndarray, node: DecisionTreeNode) -> np.ndarray:
        if isinstance(node, DecisionTreeLeaf):
            return np.full(points.shape[0], node.max_class)
        
        left_mask = points[:, node.split_dim] == 0
        right_mask = ~left_mask
        
        predictions = np.empty(points.shape[0], dtype=int)
        
        if np.any(left_mask):
            predictions[left_mask] = self._predict(points[left_mask], node.left)
        if np.any(right_mask):
            predictions[right_mask] = self._predict(points[right_mask], node.right)
        
        return predictions

    def predict(self, points: np.ndarray) -> np.ndarray:
        return self._predict(points, self._root)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [13]:
class RandomForestClassifier:

    _n_features: int = None

    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = gini if criterion == "gini" else entropy
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._max_features = max_features
        self._n_estimators = n_estimators
        self._estimators = []

    @property
    def estimators(self) -> List[DecisionTree]:
        return self._estimators

    @property
    def n_features(self) -> int:
        if self._n_features is None:
            raise RuntimeError("Fit random forest before accessing to number of features properties")
        return self._n_features

    def fit(self, X, y):
        self._n_features = X.shape[1]
        n_samples = X.shape[0]
        for _ in range(self._n_estimators):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_bootstrap = X[indices]
            y_bootstrap = y[indices]
            
            tree = DecisionTree(
                X_bootstrap, y_bootstrap,
                criterion=self.criterion,
                max_depth=self._max_depth,
                min_samples_leaf=self._min_samples_leaf,
                max_features=self._max_features
            )
            self._estimators.append(tree) 

    def predict(self, X):
        X = np.asarray(X)
        predictions = np.array([tree.predict(X) for tree in self._estimators])
        
        y_pred = np.zeros(X.shape[0], dtype=int)
        for i in range(X.shape[0]):
            (values, counts) = np.unique(predictions[:, i], return_counts=True)
            y_pred[i] = values[np.argmax(counts)]
        
        return y_pred

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [14]:
def accuracy_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    return np.mean(y_true == y_pred)


def feature_importance(rfc):
    n_features = rfc.n_features
    importance = np.zeros(n_features)
    total_trees = 0
    
    for tree in rfc.estimators:
        X_oob, y_oob = tree.out_of_bag
        
        if X_oob is None or len(X_oob) == 0:
            continue
        
        total_trees += 1
        
        y_pred = tree.predict(X_oob)
        err_oob = 1 - accuracy_score(y_oob, y_pred)
        
        for j in range(n_features):
            X_permuted = X_oob.copy()
            X_permuted[:, j] = np.random.permutation(X_permuted[:, j])
            
            y_pred_perm = tree.predict(X_permuted)
            err_oob_j = 1 - accuracy_score(y_oob, y_pred_perm)
            
            importance[j] += (err_oob_j - err_oob)
    
    if total_trees > 0:
        importance /= total_trees
    
    return importance


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    indices = np.argsort(importance)[::-1][:k]
    return np.array(names)[indices]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [15]:
def synthetic_dataset(size):
    X = [
        (np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, i % 6 == 0, i % 3 == 2, np.random.randint(0, 2))
        for i in range(size)
    ]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)


X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [-1.87670739e-04  4.37921682e-04  1.66684268e-01  1.70520656e-01
  3.30860967e-01 -2.36291108e-04]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [30]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]

    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [31]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

In [32]:
from sklearn.preprocessing import LabelEncoder

le_sex = LabelEncoder()
y_sex_train = le_sex.fit_transform(y_sex_train)
y_sex_test = le_sex.transform(y_sex_test)

le_age = LabelEncoder()
y_age_train = le_age.fit_transform(y_age_train)
y_age_test = le_age.transform(y_age_test)

#### Возраст

In [40]:
rfc = RandomForestClassifier(
    n_estimators=40,
    max_depth=None,
    max_features="auto",
    criterion="entropy",
    min_samples_leaf=1
)

rfc.fit(X_train, y_age_train)
age_pred = rfc.predict(X_test)

print("Accuracy:", np.mean(age_pred == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.7389659520807061
Most important features:
1. mudakoff
2. 4ch
3. ovsyanochan
4. styd.pozor
5. dayvinchik
6. rhymes
7. iwantyou
8. rapnewrap
9. i_des
10. bot_maxim
11. leprum
12. pravdashowtop
13. xfilm
14. tumblr_vacuum
15. fuck_humor
16. pixel_stickers
17. borsch
18. ne.poverish
19. exclusive_muzic
20. dzenpub


#### Пол

In [41]:
rfc = RandomForestClassifier(
    n_estimators=40,
    max_depth=None,
    max_features="auto",
    criterion="entropy",
    min_samples_leaf=1
)

rfc.fit(X_train, y_sex_train)
age_pred = rfc.predict(X_test)

print("Accuracy:", np.mean(age_pred == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.862547288776797
Most important features:
1. 40kg
2. mudakoff
3. zerofat
4. girlmeme
5. 9o_6o_9o
6. rapnewrap
7. be.beauty
8. modnailru
9. sh.cook
10. igm
11. cook_good
12. thesmolny
13. i_d_t
14. woman.blog
15. 4ch
16. femalemem
17. fuck_humor
18. academyofman
19. reflexia_our_feelings
20. psy.people


### CatBoost
В качестве аьтернативы попробуем CatBoost.

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [43]:
X, y = synthetic_dataset(1000)

cb_model = CatBoostClassifier(
    loss_function='MultiClass',
    iterations=10,
    learning_rate=0.01,
    depth=6
)
cb_model.fit(X, y)
y_pred = cb_model.predict(X)

print("Accuracy:", accuracy_score(y_pred, y))
print("Importance:", cb_model.feature_importances_)

0:	learn: 1.0800327	total: 802us	remaining: 7.23ms
1:	learn: 1.0613432	total: 1.47ms	remaining: 5.88ms
2:	learn: 1.0437628	total: 2.29ms	remaining: 5.33ms
3:	learn: 1.0266420	total: 2.99ms	remaining: 4.48ms
4:	learn: 1.0094097	total: 3.57ms	remaining: 3.57ms
5:	learn: 0.9926321	total: 4.14ms	remaining: 2.76ms
6:	learn: 0.9762900	total: 4.71ms	remaining: 2.02ms
7:	learn: 0.9608657	total: 5.39ms	remaining: 1.35ms
8:	learn: 0.9458149	total: 6.07ms	remaining: 674us
9:	learn: 0.9306515	total: 6.65ms	remaining: 0us
Accuracy: 1.0
Importance: [3.23821947e-05 4.56289424e-05 2.81785147e+01 2.81811671e+01
 4.36400032e+01 2.37037771e-04]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [44]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(
    X_train, y_age_train, y_sex_train, train_size=0.8
)

In [53]:
max_depth = range(1, 10, 3)
min_samples_leaf = range(1, 10, 3)
learning_rate = np.linspace(0.001, 1.0, 5)


def get_best_params(y_train, y_eval):
    best_score, best_params = -np.inf, None
    for lr, md, msl in tqdm(list(product(learning_rate, max_depth, min_samples_leaf))):
        model = CatBoostClassifier(
            learning_rate=lr,
            depth=md,
            min_data_in_leaf=msl,
            iterations=500,
            random_seed=SEED,
            verbose=False
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_eval)
        score = accuracy_score(y_eval, y_pred)
        
        if score > best_score:
            best_score, best_params = score, {
                "learning_rate": lr,
                "depth": md,
                "min_data_in_leaf": msl
            }

    return best_params, best_score

#### Возраст

In [54]:
best_params, best_score = get_best_params(y_age_train, y_age_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.25075), 'depth': 1, 'min_data_in_leaf': 1},
 np.float64(0.7512263489838823))

In [55]:
cb_model = CatBoostClassifier(
    iterations=200,
    loss_function='MultiClass',
    random_seed=SEED,
    verbose=False,
    **best_params
)
cb_model.fit(X_train, y_age_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_age_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.7187894073139974
Most important features:
1. ovsyanochan
2. 4ch
3. styd.pozor
4. rhymes
5. tumblr_vacuum
6. dayvinchik
7. mudakoff
8. in.humour
9. leprum
10. pixel_stickers


#### Пол

In [56]:
best_params, best_score = get_best_params(y_sex_train, y_sex_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.25075), 'depth': 4, 'min_data_in_leaf': 1},
 np.float64(0.8745620182200421))

In [57]:
cb_model = CatBoostClassifier(
    iterations=200,
    loss_function='MultiClass',
    random_seed=SEED,
    verbose=False,
    **best_params
)
cb_model.fit(X_train, y_sex_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_sex_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8827238335435057
Most important features:
1. 40kg
2. girlmeme
3. modnailru
4. mudakoff
5. zerofat
6. i_d_t
7. femalemem
8. cook_good
9. 9o_6o_9o
10. thesmolny
